In [ ]:
import pandas as pd
#import numpy as np
!pip install imageio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

#train_data_dir = '/content/drive/My Drive/Curso_DL_CFP/proyecto/data/train'
trainX = '/content/drive/My Drive/Curso_DL_CFP/proyecto/data_p/dataset_pandas/'
testX = '/content/drive/My Drive/Curso_DL_CFP/proyecto/data_p/examples_pandas/'
trainX_p = '/Users/albaprimoaparici/Desktop/dataset_pandas/'
testX_p = '/Users/albaprimoaparici/Desktop//examples_pandas/'


In [ ]:
import keras.applications
#from keras.applications import VGG16
from keras.applications import imagenet_utils
import keras.utils
#from keras.utils import to_categorical
from tensorflow.keras import optimizers
from keras.layers import Dropout, Flatten, Dense
import matplotlib.pyplot as plt
import keras.engine
#from keras.engine import Model
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import keras.optimizers
#from keras.optimizers import Adam
import numpy as np
from sklearn.preprocessing import LabelBinarizer
input_shape = (32, 32, 3) #(X)
labelY = ["indoor", "outdoor"]
lb = LabelBinarizer()
labelY = lb.fit_transform(labelY)
#labelY = to_categorical(labelY)

In [ ]:
# Data generators para poder hacer data augmentation
print('Usando real-time data augmentation.')
datagen_train = ImageDataGenerator( #(X)
    rotation_range=int(180*0.1),  #(X)
    width_shift_range=0.1,  #(X)
    height_shift_range=0.1,  #(X)
    zoom_range=0.1,  #(X)
    horizontal_flip=True, #(X)
    validation_split=0.2,
    rescale=1./255
    #preprocessing_function = imagenet_utils.preprocess_input
)

datagen_test = ImageDataGenerator( #(X)
    #preprocessing_function = imagenet_utils.preprocess_input
    rescale=1./255
)

In [ ]:

train_datagen = datagen_train.flow_from_directory(directory = trainX,
    target_size=(32, 32),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    follow_links=False,
    subset="training") #(X)

val_datagen = datagen_train.flow_from_directory(directory = trainX,
    target_size=(32, 32),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    follow_links=False,
    subset="validation")

test_datagen = datagen_test.flow_from_directory(directory = testX,
    target_size=(32, 32),
    class_mode="categorical",
    batch_size=32,
    shuffle=False)


In [ ]:
#Importing library
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Input, Conv2D, Activation, Flatten, Dense, Dropout, BatchNormalization, MaxPooling2D, MaxPool2D
#from keras.layers.normalization import BatchNormalization
import numpy as np

np.random.seed(1000)

#Instantiation
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(32,32,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))
#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(2))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

#Model Summary
AlexNet.summary()

# Compiling the model
#optimizer=SGD(lr=0.01, decay=0.01/50, momentum=0.9, nesterov=True)
print("[INFO]: Compilando red neuronal...")
AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

#Entrenamos
print("[INFO]: Entrenando red neuronal...")
H = AlexNet.fit(train_datagen, validation_data=val_datagen, epochs=100) 

# Montamos la unidad de Drive
drive.mount('/content/drive') #(X)
# Almacenamos el modelo empleando la función mdoel.save de Keras
AlexNet.save("/content/drive/My Drive/Curso_DL_CFP/Modelos/deepCNN_fin.h5")



In [ ]:
import cv2
import matplotlib.pyplot as plt
import imutils
import numpy as np
from google.colab import drive
from tensorflow.keras.models import load_model

labelNames = ["indoor", "outdoor"]

def predict_image(image, model):
  # Creamos una copia sobre la que mostraremos el resultado (comando image.copy())
  output = image.copy() #(X)
  # Expandimos las dimensiones (32, 32, 3) a (1, 32, 32, 3)
  image = np.expand_dims(image, axis=0) #(X)
 

  # Clasificación de la imagen empleando el modelo
  #print("[INFO]: Clasificando imagen...")
  # Realizamos la predicción
  proba = AlexNet.predict(image) #(X)
  #print(proba)
  #print (np.max(proba))
  # Nos quedamos con la clase que presente una probabilidad mayor y buscamos la etiqueta en el vector labelNames
  idx = np.argmax(proba) 
  #print(idx)#(X)
  label = labelNames[idx] 
  #print(label)#(X)
  res = label
  acc = np.max(proba)
  
  return res, acc



In [ ]:
from google.colab import drive

# Montamos la unidad de Drive
drive.mount('/content/drive') #(X)

# Selecciono imagen y la leo con OPENCV
img_path = "/content/drive/My Drive/Curso_DL_CFP/imagenes/prueba1.jpg" # Path de Drive donde tengo la imagen (incluido el nombre de la misma) #(X)
img_test = cv2.imread(img_path, cv2.IMREAD_COLOR) # Leo imagen con OPENCV
img_test = cv2.cvtColor(img_test,cv2.COLOR_BGR2RGB) # Por defecto la carga en BGR, la convierto a RGB
res = []


# Pre-procesamos tal y como he hecho para la fase de entrenamiento con las muestras de CIFAR10
img_test = img_test.astype("float") / 255.0 #(X)
# Re-escalamos la imagen al tamaño con el que fue entrenada la red (comando cv2.resize)
img_test_res = cv2.resize(img_test, (32, 32)) #(X)
# Predecimos la imagen pasando como parámetros a la función predict_image: la imagen, el modelo y string con el GT
res = predict_image(img_test_res, AlexNet)#(X)

print(res[0])
print(res[1])


In [ ]:
import cv2
import numpy as np
from google.colab import drive
from google.colab.patches import cv2_imshow
import os
# Montamos la unidad de Drive
drive.mount('/content/drive') #(X)



ImageDataPath = "/content/drive/My Drive/Curso_DL_CFP/proyecto/dataset"
label = []
post = []
acc = []
people = ''
for dir1 in os.listdir(ImageDataPath):
    print(dir1)
    if (dir1 == '.DS_Store' or dir1 == '.ipynb_checkpoints'): 
        print('borrrrraaaaaameeeeee')
    
        
    else:
        files = os.listdir(ImageDataPath + '/' + dir1)
        fotos = []
        for file in files:
            if file.endswith(".jpg"):
                original_image = cv2.imread(ImageDataPath + '/' + dir1 + '/'+ file, cv2.IMREAD_COLOR) # Leo imagen con OPENCV
                if original_image is not None:
                    # Convert image to grayscale
                    image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB) # Por defecto la carga en BGR, la convierto a RGB

                    # Pre-procesamos tal y como he hecho para la fase de entrenamiento con las muestras de CIFAR10
                    imgage = image.astype("float") / 255.0 #(X)
                    # Re-escalamos la imagen al tamaño con el que fue entrenada la red (comando cv2.resize)
                    img_res = cv2.resize(imgage, (32, 32)) #(X)
                    # Predecimos la imagen pasando como parámetros a la función predict_image: la imagen, el modelo y string con el GT
                    res = predict_image(img_res, AlexNet)#(X)

                    print(file)
                    print(res)
               
                    label.append(res[0])
                    post.append(file)
                    acc.append(res[1])
                    



                else:
                    print(f'En error occurred while trying to load image')
             


print(label)

In [ ]:
import csv 
feature_space = []
with open("/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/in_out.csv", 'w', encoding='UTF-8') as archivo:
        writer = csv.DictWriter(archivo, fieldnames=['post', 'label', 'acc'], delimiter=',')
        writer.writeheader()
        for tupla in zip(post, label, acc):
          fila = {}
          fila['post'] =  tupla[0]
          fila['label'] = tupla[1]
          fila['acc'] = tupla[2]
          feature_space.append(fila)
        
        writer.writerows(feature_space)

In [ ]:
import cv2
import numpy as np
from google.colab import drive
from google.colab.patches import cv2_imshow
# Montamos la unidad de Drive
drive.mount('/content/drive') #(X)




def draw_found_faces(detected, image, color: tuple):
    for (x, y, width, height) in detected:
        cv2.rectangle(
            image,
            (x, y),
            (x + width, y + height),
            color,
            thickness=2
        )

path_to_image="/content/drive/My Drive/Curso_DL_CFP/imagenes/prueba.png"
original_image = cv2.imread(path_to_image)
print ('1')
if original_image is not None:
    # Convert image to grayscale
    image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

    # Create Cascade Classifiers
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_profileface.xml")
    
    # Detect faces using the classifiers
    detected_faces = face_cascade.detectMultiScale(image=image, scaleFactor=1.3, minNeighbors=4)
    row, columns = np.shape(detected_faces)
    print('N º caras detectadas de frente :  ', row)
    #print(columns)
    print (detected_faces)

    detected_profiles = profile_cascade.detectMultiScale(image=image, scaleFactor=1.3, minNeighbors=4)
    row, columns = np.shape(detected_profiles)
    print('N º caras detectadas de perfil :  ', row)
    #print(columns)
    print(detected_profiles)
    # Filter out profiles
    profiles_not_faces = [x for x in detected_profiles if x not in detected_faces]
    print('4')
    # Draw rectangles around faces on the original, colored image
    draw_found_faces(detected_faces, original_image, (0, 255, 0)) # RGB - green
    draw_found_faces(detected_profiles, original_image, (0, 0, 255)) # RGB - red
    print('5')
    # Open a window to display the results
    #cv2.imshow(f'Detected Faces in {path_to_image}', original_image)
    cv2_imshow(original_image)
    # The window will close as soon as any key is pressed (not a mouse click)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()
else:
    print(f'En error occurred while trying to load {path_to_image}')

In [ ]:
import cv2
import numpy as np
from google.colab import drive
from google.colab.patches import cv2_imshow
import os
# Montamos la unidad de Drive
drive.mount('/content/drive') #(X)



ImageDataPath = "/content/drive/My Drive/Curso_DL_CFP/proyecto/dataset"
faces = []
post = []
is_people = []
people = 'false'
for dir1 in os.listdir(ImageDataPath):
    print(dir1)
    if (dir1 == '.DS_Store' or dir1 == '.ipynb_checkpoints'): 
        print('borrrrraaaaaameeeeee')
    
        
    else:
        files = os.listdir(ImageDataPath + '/' + dir1)
        fotos = []
        for file in files:
            if file.endswith(".jpg"):
                original_image = cv2.imread(ImageDataPath + '/' + dir1 + '/'+ file)
                if original_image is not None:
                    # Convertir imagen a escala de grises 
                    image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

                    # Creando clasificador
                    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    
                    # Detección de rostros usando el clasificador
                    detected_faces = face_cascade.detectMultiScale(image=image, scaleFactor=1.3, minNeighbors=4)
                    
                      try:
                        row, columns = np.shape(detected_faces)
                        print('N º caras detectadas de frente :  ', row)
                        people = 'true'
                      except (ValueError):
                        print (len(np.shape(detected_faces)))
                        row = 0
                        people = 'indeterminante'
                    
                    faces.append(row)
                    post.append(file)
                    is_people.append(people)
                    print (detected_faces)



                else:
                    print(f'En error occurred while trying to load {original_image}')
             


print(faces)

In [ ]:
print(faces)

In [ ]:
print(post)

In [ ]:
feature_faces = []
for tupla in zip(post, faces, is_people): #obtenemos la tupla en cada iteración
  print(tupla[0], tupla[1], tupla[2])
  feature_faces.append(tupla)
print(feature_faces)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import csv 
feature_faces = []
with open("/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/num_faces.csv", 'w', encoding='UTF-8') as archivo:
        writer = csv.DictWriter(archivo, fieldnames=['post', 'faces', 'is_people'], delimiter=',')
        writer.writeheader()
        for tupla in zip(post, faces, is_people):
          fila = {}
          fila['post'] =  tupla[0]
          fila['faces'] = tupla[1]
          fila['is_people'] = tupla[2]
          feature_faces.append(fila)
        
        writer.writerows(feature_faces)

In [ ]:

import cv2
from google.colab.patches import cv2_imshow
faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
image = cv2.imread('/content/drive/My Drive/Curso_DL_CFP/imagenes/prueba.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = faceClassif.detectMultiScale(gray,
  scaleFactor=1.1,
  minNeighbors=5)
for (x,y,w,h) in faces:
  cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
cv2_imshow(image)
cv2.waitKey(0)
cv2.destroyAllWindows()